In [9]:
import pandas as pd     # similar to Excel 
import numpy as np      # suitable for working with arrays
import matplotlib.pyplot as plt  # suitable for plotting
from sklearn.model_selection import train_test_split
print('Loading from repository, Done!')

def alpha(f1,f2):
    y = 1.64*(f1-f2)-2.82
    return y


def mJy_to_mag(F, F0):
    m = -2.5*np.log10(F*0.001/F0)
    return m


def e_mJy_to_mag(e_F, F):
    m = (2.5/np.log(10))*(e_F/F)
    return m


Loading from repository, Done!


In [10]:
c2d_CL = pd.read_csv("../Data_and_Results/c2d_CLOUDS_HREL.csv")
c2d_CO = pd.read_csv("../Data_and_Results/c2d_CORES_HREL.csv")
c2d_ST = pd.read_csv("../Data_and_Results/c2d_STARS_HREL.csv")
c2d_ALL_1 = pd.merge_ordered(c2d_CL,c2d_CO)
c2d = pd.merge_ordered(c2d_ST,c2d_ALL_1)


print('loading data, Done!')

loading data, Done!


In [11]:
c2d_YSO_1 = pd.merge_ordered(c2d[c2d["object_type"].str.rstrip()=='YSOc_star+dust(MP1)'],c2d[c2d["object_type"].str.rstrip()=='YSOc_star+dust(IR4)'])
c2d_YSO_2 = pd.merge_ordered(c2d[c2d["object_type"].str.rstrip()=='YSOc_star+dust(IR2)'],c2d[c2d["object_type"].str.rstrip()=='YSOc_star+dust(IR3)'])
c2d_YSO_3 = pd.merge_ordered(c2d[c2d["object_type"].str.rstrip()=='YSOc_star+dust(IR1)'],c2d[c2d["object_type"].str.rstrip()=='YSOc_red'])
c2d_YSO_4 = pd.merge_ordered(c2d[c2d["object_type"].str.rstrip()=='YSOc_PAH-em'],c2d[c2d["object_type"].str.rstrip()=='YSOc'])
c2d_YSO_12 = pd.merge_ordered(c2d_YSO_1,c2d_YSO_2)
c2d_YSO_34 = pd.merge_ordered(c2d_YSO_3,c2d_YSO_4)
c2d_YSO = pd.merge_ordered(c2d_YSO_12,c2d_YSO_34)

c2d_YSO["Target"] = 0


c2d_EG_1 = pd.merge_ordered(c2d[c2d["object_type"]=='Galc_star+dust(MP1)'],c2d[c2d["object_type"]=='Galc_star+dust(IR4)'])
c2d_EG_2 = pd.merge_ordered(c2d[c2d["object_type"]=='Galc_star+dust(IR2)'],c2d[c2d["object_type"]=='Galc_star+dust(IR3)'])
c2d_EG_3 = pd.merge_ordered(c2d[c2d["object_type"]=='Galc_star+dust(IR1)'],c2d[c2d["object_type"]=='Galc_red'])
c2d_EG_4 = pd.merge_ordered(c2d[c2d["object_type"]=='Galc_PAH-em'],c2d[c2d["object_type"]=='Galc'])
c2d_EG_12 = pd.merge_ordered(c2d_EG_1,c2d_EG_2)
c2d_EG_34 = pd.merge_ordered(c2d_EG_3,c2d_EG_4)
c2d_EG = pd.merge_ordered(c2d_EG_12,c2d_EG_34)
c2d_EG["Target"] = 1


c2d_ST_1 = pd.merge_ordered(c2d[c2d["object_type"]=='star+dust(MP1)'],c2d[c2d["object_type"]=='star+dust(IR4)'])
c2d_ST_2 = pd.merge_ordered(c2d[c2d["object_type"]=='star+dust(IR2)'],c2d[c2d["object_type"]=='star+dust(IR3)'])
c2d_ST_3 = pd.merge_ordered(c2d[c2d["object_type"]=='Galc_star+dust(IR1)'],c2d[c2d["object_type"]=='star'])
c2d_ST_12 = pd.merge_ordered(c2d_ST_1,c2d_ST_2)
c2d_ST = pd.merge_ordered(c2d_ST_3,c2d_ST_12)
c2d_ST["Target"] = 2

c2d_1 = pd.merge_ordered(c2d_YSO,c2d_EG)
c2d_NEW = pd.merge_ordered(c2d_1,c2d_ST)

print(f"YSO shape: {c2d_YSO.shape}")
print(f"EG shape: {c2d_EG.shape}")
print(f"Star shape: {c2d_ST.shape}")


YSO shape: (1463, 25)
EG shape: (3621, 25)
Star shape: (409437, 25)


In [12]:
c2d_IRAC = c2d_NEW[["object_type","region","ir1_flux_c","ir1_d_flux_c","ir2_flux_c","ir2_d_flux_c","ir3_flux_c","ir3_d_flux_c","ir4_flux_c","ir4_d_flux_c", "Target"]]


c2d_IRAC = c2d_IRAC[~c2d_IRAC["ir1_flux_c"].isna() & ~c2d_IRAC["ir2_flux_c"].isna() & ~c2d_IRAC["ir3_flux_c"].isna() & ~c2d_IRAC["ir4_flux_c"].isna()\
    & ~c2d_IRAC["ir1_d_flux_c"].isna() & ~c2d_IRAC["ir2_d_flux_c"].isna() & ~c2d_IRAC["ir3_d_flux_c"].isna() & ~c2d_IRAC["ir4_d_flux_c"].isna()]

# Load in magnitude data
# Access data by band instead of object
c2d_IRAC_fluxes = c2d_IRAC[["ir1_flux_c","ir1_d_flux_c","ir2_flux_c","ir2_d_flux_c","ir3_flux_c","ir3_d_flux_c","ir4_flux_c","ir4_d_flux_c"]].values.astype(float).transpose() 
band_names = ["ir1_flux_c","ir1_d_flux_c","ir2_flux_c","ir2_d_flux_c","ir3_flux_c","ir3_d_flux_c","ir4_flux_c","ir4_d_flux_c"]
# Convert magnitudes to flux
for i, F_0 in enumerate([277.500, 277.500, 179.500, 179.500, 116.600, 116.600, 63.100, 63.100]):
    if i%2==0:
        c2d_IRAC[band_names[i]] = mJy_to_mag(c2d_IRAC_fluxes[i],F_0) # Convert fluxes in a column to mags
    else:
        c2d_IRAC[band_names[i]] = e_mJy_to_mag(c2d_IRAC_fluxes[i],c2d_IRAC_fluxes[i-1]) # Convert errors in a column to mag
inf_ind_2 = c2d_IRAC[["ir1_flux_c","ir1_d_flux_c","ir2_flux_c","ir2_d_flux_c","ir3_flux_c","ir3_d_flux_c","ir4_flux_c","ir4_d_flux_c"]].index[np.isinf(c2d_IRAC[["ir1_flux_c","ir1_d_flux_c","ir2_flux_c","ir2_d_flux_c","ir3_flux_c","ir3_d_flux_c","ir4_flux_c","ir4_d_flux_c"]]).any(1)]
c2d_IRAC = c2d_IRAC.drop(index = inf_ind_2)
c2d_IRAC["alpha"] = alpha(c2d_IRAC["ir2_flux_c"],c2d_IRAC["ir4_flux_c"])

c2d_IRAC = c2d_IRAC[~c2d_IRAC["ir1_flux_c"].isna() & ~c2d_IRAC["ir2_flux_c"].isna() & ~c2d_IRAC["ir3_flux_c"].isna() & ~c2d_IRAC["ir4_flux_c"].isna()\
    & ~c2d_IRAC["ir1_d_flux_c"].isna() & ~c2d_IRAC["ir2_d_flux_c"].isna() & ~c2d_IRAC["ir3_d_flux_c"].isna() & ~c2d_IRAC["ir4_d_flux_c"].isna() & ~c2d_IRAC["alpha"].isna()]

print("Cleaning data, done!")

Cleaning data, done!


/var/folders/qy/7yz8bmv96hx8rndql_rfdz_80000gn/T/ipykernel_55262/3672380420.py:13: RuntimeWarning: divide by zero encountered in log10
  m = -2.5*np.log10(F*0.001/F0)
/var/folders/qy/7yz8bmv96hx8rndql_rfdz_80000gn/T/ipykernel_55262/3672380420.py:13: RuntimeWarning: invalid value encountered in log10
  m = -2.5*np.log10(F*0.001/F0)
/var/folders/qy/7yz8bmv96hx8rndql_rfdz_80000gn/T/ipykernel_55262/3672380420.py:18: RuntimeWarning: invalid value encountered in true_divide
  m = (2.5/np.log(10))*(e_F/F)
/var/folders/qy/7yz8bmv96hx8rndql_rfdz_80000gn/T/ipykernel_55262/3672380420.py:18: RuntimeWarning: divide by zero encountered in true_divide
  m = (2.5/np.log(10))*(e_F/F)


In [24]:
c2d = c2d_IRAC[["ir1_flux_c","ir1_d_flux_c","ir2_flux_c","ir2_d_flux_c","ir3_flux_c","ir3_d_flux_c","ir4_flux_c","ir4_d_flux_c","alpha","Target"]]
c2d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 343581 entries, 7 to 414520
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   ir1_flux_c    343581 non-null  float64
 1   ir1_d_flux_c  343581 non-null  float64
 2   ir2_flux_c    343581 non-null  float64
 3   ir2_d_flux_c  343581 non-null  float64
 4   ir3_flux_c    343581 non-null  float64
 5   ir3_d_flux_c  343581 non-null  float64
 6   ir4_flux_c    343581 non-null  float64
 7   ir4_d_flux_c  343581 non-null  float64
 8   alpha         343581 non-null  float64
 9   Target        343581 non-null  int64  
dtypes: float64(9), int64(1)
memory usage: 28.8 MB


In [26]:
cat_t= c2d.values

np.shape(cat_t)

(343581, 10)

In [27]:
cat, cat_v = train_test_split(cat_t,train_size=.80,random_state=0)
print (np.shape(cat),np.shape(cat_v))

(274864, 10) (68717, 10)


In [30]:
idx_0v=(np.where(cat_v[:,9]==0)[0])
idx_1v=(np.where(cat_v[:,9]==1)[0])
idx_2v=(np.where(((cat_v[:,9]!=1) & (cat_v[:,9]!=0)))[0])

cat_v[idx_0v,9]=0
cat_v[idx_1v,9]=1
cat_v[idx_2v,9]=2

inp_test= cat_v[:,[0,2,4,6]]
tar_test= cat_v[:,9]

print (len(cat),len(cat_v),len(cat_t))


274864 68717 343581


In [31]:
len(tar_test)

68717

In [39]:
len(np.where(cat[:,9]==1)[0])

2890

In [40]:
idx_0=(np.where(cat[:,9]==0)[0])
idx_1=(np.where(cat[:,9]==1)[0])
idx_2=(np.where((cat[:,9]!=1) & (cat[:,9]!=0))[0])

print (len(idx_0),len(idx_1),len(idx_2))

1149 2890 270825


In [42]:
n_sigma=1


sig_36=n_sigma* np.percentile(cat[:,1],68)
sig_45=n_sigma*np.percentile(cat[:,3],68)
sig_58=n_sigma*np.percentile(cat[:,5],68)
sig_80=n_sigma*np.percentile(cat[:,7],68)
sig_36

0.07025351913140836

In [43]:
mag= cat[:,[0,2,4,6]]
err= cat[:,[1,3,5,7]]



In [44]:
mag_0=mag[idx_0]  # 414
mag_1=mag[idx_1]  # 2659
mag_2=mag[idx_2]  # 23830


err_0=err[idx_0]  # 414
err_1=err[idx_1]  # 2659
err_2=err[idx_2]  # 23830

np.shape(mag_0)

(1149, 4)

In [14]:
# n_sample=10
# inp=[]
# tar=[]
# for k1 in range(1):
#     bts_0=np.random.choice(len(mag_0),n_sample,replace=True)
#     bts_1=np.random.choice(len(mag_1),n_sample,replace=True)
#     bts_2=np.random.choice(len(mag_2),n_sample,replace=True)
#     #mag_2[bts_2]
#     #np.abs(np.random.randn(n_sample))
#     print(mag_2[bts_2])
    


In [45]:


def BST(mag,n_sample,n_sig=1, sig_36=sig_36,sig_45=sig_45,sig_58=sig_58,sig_80=sig_80):
    bts=np.random.choice(len(mag),n_sample,replace=True)
    err=np.random.randn(n_sample,4)
    mag_bts= mag[bts]
    err[:,0]*=sig_36*n_sig
    err[:,1]*=sig_45*n_sig
    err[:,2]*=sig_58*n_sig
    err[:,3]*=sig_80*n_sig
    mag_dist= mag_bts+ err
    return mag_dist
    


array([[10.93972129, 10.86724813, 11.30979302,  5.25672749],
       [ 9.03922288,  7.72935252,  7.45833996,  3.62999515],
       [ 9.97955174,  8.67170516,  7.89272602,  7.51548285],
       [ 9.41026083,  9.35861581, 10.50617733,  4.81102513],
       [13.1393057 , 12.65505628, 11.67149809, 10.67801683],
       [ 8.8372322 ,  8.41727557,  8.28206352,  7.89045604],
       [ 8.86270096,  8.39925525,  7.46376208,  8.83079066],
       [13.04706543, 12.69565901, 10.91249475, 11.56571762],
       [ 7.27415874,  6.60017293,  6.11637038,  8.67246274],
       [ 6.85432947,  6.71946626,  6.77142576,  3.9526895 ]])

In [47]:
n_sample=25
n_sig = 1
inp=[]
tar=[]

for k1 in range(int(10000/n_sample)):
    mag0= BST(mag_0,n_sample=n_sample,n_sig=n_sig)
    tar.append([0]*n_sample)
    inp.append(mag0) 
    mag1= BST(mag_1,n_sample=n_sample,n_sig=n_sig)
    tar.append([1]*n_sample)
    inp.append(mag1) 
    mag2= BST(mag_2,n_sample=n_sample,n_sig=n_sig)
    tar.append([2]*n_sample)
    inp.append(mag2) 


 
            


inp = np.array(inp)
inp=np.reshape(inp, (np.shape(inp)[0]*np.shape(inp)[1],4))
tar=np.reshape(tar,-1)

n_arr=np.arange(len(inp))
np.random.shuffle(n_arr)
inp= inp[n_arr]
tar= tar[n_arr]


print(np.shape(inp),np.shape(tar))

(30000, 4) (30000,)


In [50]:
np.save('../Data_and_Results/c2d_ALL_HT_Inputs',inp)
np.save('../Data_and_Results/c2d_ALL_HT_Targets',tar)
np.save('../Data_and_Results/c2d_ALL_HT_mag0',mag_0)
np.save('../Data_and_Results/c2d_ALL_HT_mag1',mag_1)
np.save('../Data_and_Results/c2d_ALL_HT_mag2',mag_2)

np.save('../Data_and_Results/c2d_ALL_HT_inp_test',inp_test)
np.save('../Data_and_Results/c2d_ALL_HT_tar_test',tar_test)
print('Done')

Done
